In [1]:
import pandas as pd

In [5]:
import yfinance as yf
import pandas as pd

# Load the uploaded CSV file to examine its structure
file_path = 'Book3.csv'
book3_df = pd.read_csv(file_path)

# Extract stock symbols from the DataFrame
stock_symbols = book3_df.columns

# Function to fetch historical data
def fetch_historical_data(symbol, period="1y"):
    ticker = yf.Ticker(symbol+".NS")
    hist = ticker.history(period=period)
    return hist

# Dictionary to store historical data
historical_data = {}

exception_list = []

# Fetch historical data for each stock symbol
for symbol in stock_symbols:
    try:
        ticker = yf.Ticker(symbol+".NS")
        hist = ticker.history(period="1y")
    except Exception as e:
        exception_list.append(symbol)
        print(f"Could not fetch data for {symbol}: {e}")

# Convert outstanding shares data to long format for merging
outstanding_shares_df = book3_df.melt(id_vars=['Unnamed: 0.2'], var_name='Symbol', value_name='Outstanding_Shares')
outstanding_shares_df.rename(columns={'Unnamed: 0.2': 'Year'}, inplace=True)

# Create a list to store processed data
processed_data = []

# Process each stock symbol
for symbol in stock_symbols:
    if symbol in historical_data:
        hist_df = historical_data[symbol]
        hist_df['Year'] = hist_df.index.year
        outstanding_shares = outstanding_shares_df[outstanding_shares_df['Symbol'] == symbol]

        # Merge historical data with outstanding shares data
        merged_df = hist_df.reset_index().merge(outstanding_shares, on='Year', how='left')
        merged_df['Turnover'] = merged_df['Volume'] / merged_df['Outstanding_Shares']
        
        # Append to processed data list
        processed_data.append(merged_df)

# Concatenate all processed data
final_df = pd.concat(processed_data, ignore_index=True)

# Save the final DataFrame to CSV
final_df.to_csv('/mnt/data/final_turnover_data.csv', index=False)

# Display the first few rows of the final DataFrame
final_df.head()


In [30]:
import pandas as pd
import yfinance as yf
from datetime import datetime

# Load the provided CSV file
file_path = 'Book3.csv'
data = pd.read_csv(file_path)

# Define the date range to end at the last day of 2023
end_date = datetime(2024, 1, 1)
start_date = end_date - pd.DateOffset(years=10)

# Initialize a list to keep track of any errors and a list to store the merged data
errors = []
merged_data_list = []

# Process each stock in the data (excluding the 'YEAR' column)
for ticker in data.columns[1:]:
    try:
        # Fetch historical data for the stock
        stock_data = yf.download(f"{ticker}.NS", start=start_date, end=end_date)
        
        # Round the stock data to one decimal place
        stock_data = stock_data.round(1)
        
        # Add additional columns to the stock data
        stock_data['Date'] = stock_data.index
        stock_data['Symbol'] = ticker
        stock_data['Year'] = stock_data.index.year
        
        # Drop the High, Low, and Close columns
        stock_data.drop(columns=['High', 'Low', 'Open', 'Close'], inplace=True)
        
        # Merge the outstanding shares data with the stock data for each year
        outstanding_shares_data = data[['YEAR', ticker]]
        outstanding_shares_data.columns = ['Year', 'Outstanding_Shares']
        
        # Merge the data based on the Year column while retaining the index
        merged_data = stock_data.merge(outstanding_shares_data, on='Year', how='left')
        
        # Append the merged data to the list
        merged_data_list.append(merged_data)
    except Exception as e:
        errors.append((ticker, str(e)))

# Combine all the merged data into a single DataFrame
combined_data = pd.concat(merged_data_list)

# Save the combined data to a new CSV file
combined_data.to_csv('combined_stock_data.csv', index=False)

# Print errors if any occurred
print(errors)


In [35]:
data = pd.read_csv('combined_stock_data.csv')

data.groupby('Symbol')['Date'].count()

In [33]:
stock_data = yf.download("SAFARI.NS", start=start_date, end=end_date)

In [36]:
# Step 1: Count the occurrences of each symbol
symbol_counts = data.groupby('Symbol')['Date'].count()

# Step 2: Filter symbols with at least 2465 occurrences
valid_symbols = symbol_counts[symbol_counts >= 2465].index

# Step 3: Filter the original DataFrame to include only valid symbols
filtered_data = data[data['Symbol'].isin(valid_symbols)]

In [40]:
filtered_data.to_csv('final_data.csv')